In [1]:
import os
import aisdb
from aisdb.database.dbconn import SQLiteDBConn
from aisdb import DBConn, DBQuery, DomainFromPoints
from datetime import datetime
import nest_asyncio
nest_asyncio.apply()

## Download bathymetry data

In [2]:
# set the path to the data storage directory
bathymetry_data_dir = "./bathymetry_data/"

# check if the directory exists
if not os.path.exists(bathymetry_data_dir):
    os.makedirs(bathymetry_data_dir)

# check if the directory is empty\
if os.listdir(bathymetry_data_dir) == []:
    # download the bathymetry data
    bathy = aisdb.webdata.bathymetry.Gebco(data_dir=bathymetry_data_dir)
    bathy.fetch_bathymetry_grid()
else:
    print("Bathymetry data already exists.")

./bathymetry_data/raster-bathy.7z: 100%|███▉| 2.28G/2.28G [00:20<00:00, 109MB/s]


Extracting bathymetric data...


## Integrate AIS tracks with bathymetry raster file

In [4]:
dbpath = 'YOUR_DATABASE.db' # define the path to your database
start_time = datetime.strptime("2018-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2018-01-02 00:00:00", '%Y-%m-%d %H:%M:%S')
domain = DomainFromPoints(points=[(-63.6, 44.6)], radial_distances=[500000])

# define a function to add color to the tracks
def add_color(tracks):
    for track in tracks:
        # Calculate the average coastal distance
        avg_coast_distance = sum(abs(dist) for dist in track['coast_distance']) / len(track['coast_distance'])
        
        # Determine the color based on the average coastal distance
        if avg_coast_distance <= 100:
            track['color'] = "yellow"
        elif avg_coast_distance <= 1000:
            track['color'] = "orange"
        elif avg_coast_distance <= 20000:
            track['color'] = "pink"
        else:
            track['color'] = "red"
        
        yield track

with SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        xmin=domain.boundary['xmin'], xmax=domain.boundary['xmax'],
        ymin=domain.boundary['ymin'], ymax=domain.boundary['ymax'],
        callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi,
    )
    tracks = aisdb.track_gen.TrackGen(qry.gen_qry(), decimate=False)

    # merge the tracks with the raster data
    raster_path = "./bathymetry_data/gebco_2022_n90.0_s0.0_w-90.0_e0.0.tif" # You need to specify the geographical region to overlap based on the location of your vessel tracks
    raster = aisdb.webdata.load_raster.RasterFile(raster_path)
    tracks_raster = raster.merge_tracks(tracks, new_track_key="coast_distance")

    # add color to the tracks
    tracks_colored = add_color(tracks_raster)

    if __name__ == '__main__':
        aisdb.web_interface.visualize(
            tracks_colored,
            visualearth=True,
            open_browser=True,
        )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:43612 - received: {'msgtype': 'meta'}
127.0.0.1:43624 - received: {'msgtype': 'validrange'}
127.0.0.1:43624 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...
